In [1]:
## Tested using the following loaded modules
## module use /g/data/access/ngm/modules
## module load analysis3/21.10
import os, sys
workdir = "/g/data/xv83/users/bxn599/ACS/evaluation/"
os.chdir(workdir)
sys.path.append("./lib")
import glob
import xarray as xr
import numpy as np
import geopandas as gp
from datetime import datetime as dt
import lib_standards
import lib_spatial
import spatial_selection
import matplotlib as mpl
import matplotlib.pyplot as plt
import importlib
importlib.reload(lib_standards)
import warnings
warnings.filterwarnings('ignore')
import csv
import ccam_drs_interface
import re

font = {'size'   : 12}
mpl.rc('font', **font)

In [2]:
DATA_LOCATIONS_HIST = {
    'CCAM:ACCESS-CM2': "", # not yet post processed
    'CCAM:ACCESS-ESM1-5': "", # not yet post processed
    'CCAM:CESM2': "", # not yet post processed
    'CCAM:CMCC-ESM2': "", # not yet post processed
    'CCAM:CNRM-ESM2-1': "", # not yet post processed
    'CCAM:EC-Earth3': "/g/data/xv83/users/bxn599/ACS/icclim_indices/AUS-10/CSIRO/EC-Earth-Consortium-EC-Earth3/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/climdex",
    'CCAM:MPI-ESM1-2-HR': "", # not yet run
    'CCAM:NorESM2-MM': "/g/data/xv83/users/bxn599/ACS/icclim_indices/AUS-10/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/climdex",
}
DATA_LOCATIONS_FUT = {
    'CCAM:ACCESS-CM2': "", # not yet post processed
    'CCAM:ACCESS-ESM1-5': "", # not yet post processed
    'CCAM:CESM2': "", # not yet post processed
    'CCAM:CMCC-ESM2': "", # not yet post processed
    'CCAM:CNRM-ESM2-1': "", # not yet post processed
    'CCAM:EC-Earth3': "/g/data/xv83/users/bxn599/ACS/icclim_indices/AUS-10/CSIRO/EC-Earth-Consortium-EC-Earth3/ssp370/r1i1p1f1/CSIRO-CCAM-2203/v1/climdex",
    'CCAM:MPI-ESM1-2-HR': "", # not yet run
    'CCAM:NorESM2-MM': "/g/data/xv83/users/bxn599/ACS/icclim_indices/AUS-10/CSIRO/NCC-NorESM2-MM/ssp370/r1i1p1f1/CSIRO-CCAM-2203/v1/climdex",
}
# Frequency of icclim indicator
FREQ = 'month'
# Data sources to consider here
SOURCES_HIST = ['CCAM:EC-Earth3', 'CCAM:NorESM2-MM']

SOURCES_FUT = SOURCES_HIST

# Grid to used as the reference grid to compute difference metrics
REFERENCE_GRID = 'CCAM:NorESM2-MM'
# "Truth" for compute difference metrics
REFERENCE = 'AGCD'
# Region of interest, as per defined in lib_standards.DOMAINS
REGION = 'Australia'  
# Period of interest, as per defined in lib_standards.PERIODS
EARLYPERIOD = 'ACS_HISTORICAL' 
LATEPERIOD = 'ACS_2020'
SEASON='all'

In [3]:
def prepare_data_hist(index, sources=SOURCES_HIST, freq=FREQ, region=REGION, period=EARLYPERIOD, reference_grid=REFERENCE_GRID, season=SEASON):
    """
    Reads in the icclim data and regrid to the reference grid.
    """
    # Load the data sources
    ds = {}
    for s in sources:
        fs = glob.glob(os.path.join(DATA_LOCATIONS_HIST[s], index, "{index}_*_{freq}_*.nc".format(index=index, freq=freq)))
        ds[s] = xr.open_dataset(fs[0])
        ds[s] = lib_standards.standardise_data(ds[s], region=region, period=period, season=season, compute=True)
        experiment=re.findall('(?<=historical/)(.*)(?=/CSIRO-CCAM)',DATA_LOCATIONS_HIST[s])
    return ds, experiment

In [4]:
def prepare_data_fut(index, sources=SOURCES_FUT, freq=FREQ, region=REGION, period=LATEPERIOD, reference_grid=REFERENCE_GRID, season=SEASON):
    """
    Reads in the icclim data and regrid to the reference grid.
    """
    # Load the data sources
    ds = {}
    for s in sources:
        fs = glob.glob(os.path.join(DATA_LOCATIONS_FUT[s], index, "{index}_*_{freq}_*.nc".format(index=index, freq=freq)))
        ds[s] = xr.open_dataset(fs[0])
        ds[s] = lib_standards.standardise_data(ds[s], region=region, period=period, season=season, compute=True)
    return ds

In [5]:
header = ['Season', 'Variable', 'Project', 'Domain', 'Downscaler', 'Region', 'Model', 'Experiment', 'Variant', 'Hist-Epoch', 'Proj-Epoch', 'Change-Type', 'Value']
f = open('/g/data/xv83/users/bxn599/ACS/evaluation/test.csv','w')
writer = csv.writer(f)
writer.writerow(header)
INDEXES = ['PRCPTOT', 'PRCPTOT:DJF', 'PRCPTOT:JJA', 'PRCPTOT:MAM', 'PRCPTOT:SON']
LATEPERIOD = {'ACS_2020': '2020-2039', 'ACS_2040': '2040-2069', 'ACS_2060': '2060-2079', 'ACS_2080': '2080-2099'}
for PERIOD in LATEPERIOD:
    for index in INDEXES:
        futperiod=LATEPERIOD[PERIOD]
        print("index = {:}".format(index))
        # check whether the index has special conditions
        index_condition = None
        if not ":" in index:
            # Prepare all the icclim data as per the setup in CONFIGURATIONS
            ds0, experiment = prepare_data_hist(index)
            ds1 = prepare_data_fut(index, period=PERIOD)
            clabel = "% change of {index} ({freq})".format(index=index, freq=FREQ)
            season = 'annual'
        else:
            index_condition = index.split(":")[1]
            index = index.split(":")[0]
            ds0, experiment = prepare_data_hist(index, season=index_condition)
            ds1 = prepare_data_fut(index, season=index_condition, period=PERIOD)
            clabel = "% change of {index} ({index_condition})".format(index=index, index_condition=index_condition)
            season = index_condition

        # print(futperiod)
        # print(model)
        # print(experiment[0])
        # print(season)
        ds0_target = {}
        ds1_target = {}
        ds_target = {}
        flist = []
        for s0, s1 in zip(SOURCES_HIST, SOURCES_FUT):
            print(s0, s1)
            model=s0.split(":")[1]
            # print(model)
            # print("test")
            ds0_target[s0] = ds0[s0][index].mean(dim='time')
            ds0_target[s0] = lib_spatial.apply_region_mask(ds0_target[s0], 'Australia')
            ds1_target[s1] = ds1[s1][index].mean(dim='time')
            ds1_target[s1] = lib_spatial.apply_region_mask(ds1_target[s1], 'Australia')

            ds_target[s0] = (ds1_target[s1]-ds0_target[s0])/ds0_target[s0]*100 # percentage change
            #print(ds_target[s0])
            weights = np.cos(np.deg2rad(ds_target[s0].lat))
            weights.name = "weights"
            area_avg = ds_target[s0].weighted(weights)
            area_avg = ds_target[s0].mean(("lon", "lat"))
            print(area_avg.values)
            data = [season, 'pr', 'CCAM', 'AUS-10i', 'n/a', 'Australia', model, 'ssp370', experiment[0], '1995-2014', futperiod, 'percent',area_avg.values]
            writer.writerow(data)

index = PRCPTOT
CCAM:EC-Earth3 CCAM:EC-Earth3
13.044092824297751
CCAM:NorESM2-MM CCAM:NorESM2-MM
-10.438130924068329
index = PRCPTOT:DJF
CCAM:EC-Earth3 CCAM:EC-Earth3
13.015699674940672
CCAM:NorESM2-MM CCAM:NorESM2-MM
-10.499936412246186
index = PRCPTOT:JJA
CCAM:EC-Earth3 CCAM:EC-Earth3
17.082925459288365
CCAM:NorESM2-MM CCAM:NorESM2-MM
-4.523396117582965
index = PRCPTOT:MAM
CCAM:EC-Earth3 CCAM:EC-Earth3
19.217967955717537
CCAM:NorESM2-MM CCAM:NorESM2-MM
-5.255890952171506
index = PRCPTOT:SON
CCAM:EC-Earth3 CCAM:EC-Earth3
8.893193645582615
CCAM:NorESM2-MM CCAM:NorESM2-MM
-14.46529386061059
index = PRCPTOT
CCAM:EC-Earth3 CCAM:EC-Earth3
10.907735600741537
CCAM:NorESM2-MM CCAM:NorESM2-MM
-5.791857014525986
index = PRCPTOT:DJF
CCAM:EC-Earth3 CCAM:EC-Earth3
16.076344400138822
CCAM:NorESM2-MM CCAM:NorESM2-MM
1.0940364833333314
index = PRCPTOT:JJA
CCAM:EC-Earth3 CCAM:EC-Earth3
7.840861933831836
CCAM:NorESM2-MM CCAM:NorESM2-MM
-11.172560017711202
index = PRCPTOT:MAM
CCAM:EC-Earth3 CCAM:EC-Eart

In [6]:
INDEXES = ['TG', 'TG:DJF', 'TG:JJA', 'TG:MAM', 'TG:SON']
LATEPERIOD = {'ACS_2020': '2020-2039', 'ACS_2040': '2040-2069', 'ACS_2060': '2060-2079', 'ACS_2080': '2080-2099'}
for PERIOD in LATEPERIOD:
    for index in INDEXES:
        futperiod=LATEPERIOD[PERIOD]
        print("index = {:}".format(index))
        # check whether the index has special conditions
        index_condition = None
        if not ":" in index:
            # Prepare all the icclim data as per the setup in CONFIGURATIONS
            ds0, experiment = prepare_data_hist(index)
            ds1 = prepare_data_fut(index, period=PERIOD)
            clabel = "% change of {index} ({freq})".format(index=index, freq=FREQ)
            season = 'annual'
        else:
            index_condition = index.split(":")[1]
            index = index.split(":")[0]
            ds0, experiment = prepare_data_hist(index, season=index_condition)
            ds1 = prepare_data_fut(index, season=index_condition, period=PERIOD)
            clabel = "% change of {index} ({index_condition})".format(index=index, index_condition=index_condition)
            season = index_condition

        # print(futperiod)
        # print(model)
        # print(experiment[0])
        # print(season)
        ds0_target = {}
        ds1_target = {}
        ds_target = {}
        flist = []
        for s0, s1 in zip(SOURCES_HIST, SOURCES_FUT):
            print(s0, s1)
            model=s0.split(":")[1]
            # print(model)
            # print("test")
            ds0_target[s0] = ds0[s0][index].mean(dim='time')
            ds0_target[s0] = lib_spatial.apply_region_mask(ds0_target[s0], 'Australia')
            ds1_target[s1] = ds1[s1][index].mean(dim='time')
            ds1_target[s1] = lib_spatial.apply_region_mask(ds1_target[s1], 'Australia')

            ds_target[s0] = (ds1_target[s1]-ds0_target[s0]) # absolute change
            #print(ds_target[s0])
            weights = np.cos(np.deg2rad(ds_target[s0].lat))
            weights.name = "weights"
            area_avg = ds_target[s0].weighted(weights)
            area_avg = ds_target[s0].mean(("lon", "lat"))
            print(area_avg.values)
            data = [season, 'tg', 'CCAM', 'AUS-10i', 'n/a', 'Australia', model, 'ssp370', experiment[0], '1995-2014', futperiod, 'absolute', area_avg.values]
            writer.writerow(data)

index = TG
CCAM:EC-Earth3 CCAM:EC-Earth3
0.5861393
CCAM:NorESM2-MM CCAM:NorESM2-MM
0.4982395
index = TG:DJF
CCAM:EC-Earth3 CCAM:EC-Earth3
0.5390011
CCAM:NorESM2-MM CCAM:NorESM2-MM
0.8168269
index = TG:JJA
CCAM:EC-Earth3 CCAM:EC-Earth3
0.8450431
CCAM:NorESM2-MM CCAM:NorESM2-MM
0.30108622
index = TG:MAM
CCAM:EC-Earth3 CCAM:EC-Earth3
0.3343079
CCAM:NorESM2-MM CCAM:NorESM2-MM
0.37431526
index = TG:SON
CCAM:EC-Earth3 CCAM:EC-Earth3
0.6262053
CCAM:NorESM2-MM CCAM:NorESM2-MM
0.50072974
index = TG
CCAM:EC-Earth3 CCAM:EC-Earth3
1.2452685
CCAM:NorESM2-MM CCAM:NorESM2-MM
1.1925055
index = TG:DJF
CCAM:EC-Earth3 CCAM:EC-Earth3
1.1359639
CCAM:NorESM2-MM CCAM:NorESM2-MM
1.261571
index = TG:JJA
CCAM:EC-Earth3 CCAM:EC-Earth3
1.3867127
CCAM:NorESM2-MM CCAM:NorESM2-MM
0.71777576
index = TG:MAM
CCAM:EC-Earth3 CCAM:EC-Earth3
1.2287297
CCAM:NorESM2-MM CCAM:NorESM2-MM
1.275567
index = TG:SON
CCAM:EC-Earth3 CCAM:EC-Earth3
1.2296679
CCAM:NorESM2-MM CCAM:NorESM2-MM
1.5151081
index = TG
CCAM:EC-Earth3 CCAM:EC-Ea

In [7]:
f.close()